In [ ]:
import ast
import collections
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import sqlite3

In [ ]:
class SQLiteContext:
    def __init__(self):
        self.conn = sqlite3.connect('/home/ghsong/django-substring/mysite/db.sqlite3')
        self.conn.row_factory = dict_factory
    
    def __enter__(self):
        return self.conn.cursor()
    
    def __exit__(self, type, value, traceback):
        self.conn.close()

In [ ]:
def execute_query(query):
    with SQLiteContext() as cur:
        cur.execute(query)
        return cur.fetchall()

In [ ]:
query_tmpl = 'SELECT exp_date, algorithm_id, dataset_id, parameter, result FROM projectManager_expitem WHERE project_id=1 {} ORDER BY -exp_date'

In [ ]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [ ]:
def str2dict(s):
    return ast.literal_eval(s)

In [ ]:
setting_list = ['NoFilter', 'IF', 'NaivePF', 'ICF', 'IPF', 'LF', 'PF']
dict_title = {
    'NoFilter':'LRSim-NAIVE',
    'IF':'LRSim-INDEX',
    'NaivePF':'LRSim-PKDUCK',
    'ICF':'LRSim-COUNT',
    'IPF':'LRSim-POS',
    'LF':'LRSim-LEN',
    'PF':'LRSim-PREFIX',
             }
def get_setting_label(bLF, bPF, idx_impl):
    if not bLF and not bPF and idx_impl == 'None': return 'NoFilter'
    elif not bLF and not bPF and idx_impl == 'Naive': return 'IF'
    elif not bLF and not bPF and idx_impl == 'Count': return 'ICF'
    elif not bLF and not bPF and idx_impl == 'Position': return 'IPF'
    elif bLF and not bPF and idx_impl == 'Position': return 'LF'
    elif bLF and bPF and idx_impl == 'Position': return 'PF'
    elif bLF and bPF and idx_impl == 'None': return 'NoIndex'
    elif bLF and bPF and idx_impl == 'Naive': return 'NaivePF'
    else: return None

In [ ]:
alg_name_final = 'PrefixSearch_6.17'

In [ ]:
dict_alg = {}
dict_aid = {}
with SQLiteContext() as cur:
    cur.execute("SELECT id, name, version FROM projectManager_algorithm WHERE project_id=1")
    rows = cur.fetchall()
    for row in rows: 
        dict_alg[row['name']+'_'+row['version']] = row['id']
        dict_aid[row['id']] = row['name']+'_'+row['version']

In [ ]:
DataInfo = collections.namedtuple('DataInfo', ['name', 'size', 'nr', 'qlen'])

In [ ]:
dict_data = {}
dict_did = {}
with SQLiteContext() as cur:
    cur.execute("SELECT id, name FROM projectManager_dataset WHERE project_id=1 and id >= 71")
    rows = cur.fetchall()
    for row in rows: 
        dname, size, nr, qlen = row['name'].rsplit('_', 3)
        size, nr, qlen = size[1:], nr[1:], qlen[1:]
        try: dict_data[dname]
        except: dict_data[dname] = {}
        dict_data[dname][(size, nr, qlen)] = row['id']
        dict_did[row['id']] = (dname, size, nr, qlen)

In [ ]:
def get_did_list(data_info):
    assert type(data_info) == DataInfo
    dname, size, nr, qlen = data_info
    const_idx_list = list(map(lambda x:x[0], filter(lambda x: x[1]!='*', enumerate([size, nr, qlen]))))
    assert len(const_idx_list) == 2
    target_idx = [size, nr, qlen].index('*')
    data_item_list = dict_data[dname].items()
    for idx in const_idx_list:
        data_item_list = list(filter(lambda x: x[0][idx] == data_info[idx+1], data_item_list))
    return list(sorted(map(lambda x: (x[0][target_idx], x[1]), data_item_list), key=lambda x:int(x[0])))

In [ ]:
def get_size_did_list(dname, nr, qlen):
    return list(
        sorted(
            map(lambda x: (x[0][0], x[1]), 
                filter(lambda x: x[0][1]==nr and x[0][2]==qlen, dict_data[dname].items()))
        , key=lambda x: int(x[0])))

In [ ]:
def get_qlen_did_list(dname, size, nr):
    return list(
        sorted(
            map(lambda x: (x[0][2], x[1]), 
                filter(lambda x: x[0][0]==size and x[0][1]==nr, dict_data[dname].items()))
        , key=lambda x: int(x[0])))

In [ ]:
# marker_list = itertools.cycle(['+', 'x', 'o', '^', 'D', 's', 'v'])
marker_list = ['+', 'x', 'o', '^', 'D', 's', 'v']
dashes_list = itertools.cycle([0])

In [ ]:
def parse_data_info(data_info):
    assert type(data_info) == DataInfo
    dname, size, nr, qlen = data_info
    return dname, size, nr, qlen

In [ ]:
def df_time_by_filtering(data_info, alg_name=alg_name_final):
    dname, size, nr, qlen = parse_data_info(data_info)
    did = dict_data[dname][(size, nr, qlen)]
    aid = dict_alg[alg_name]
    predicates = ' '.join(['AND dataset_id={}'.format(did), 'AND algorithm_id={}'.format(aid)])
    query = query_tmpl.format(predicates)
    output_list = []
    for row in execute_query(query):
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])
        output = {}
        output['algorithm_id'] = row['algorithm_id']
        output['theta'] = float(dict_param['theta'])
        bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
        idx_impl = dict_param['index_impl']
        output['setting'] = get_setting_label(bLF, bPF, idx_impl)
        if output['setting'] is None: continue
        output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
        output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
        output['Num_QS_Verified'] = int(dict_rslt['Num_QS_Verified'])
        output['Num_TS_Verified'] = int(dict_rslt['Num_TS_Verified'])
        output['Num_Verified'] = int(dict_rslt['Num_QS_Verified']) + int(dict_rslt['Num_TS_Verified'])
        output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_size(data_info, alg_name=alg_name_final):
    dname, size, nr, qlen = parse_data_info(data_info)
    size_did_list = get_did_list(data_info)
    aid = dict_alg[alg_name]
    output_list = []
    for size, did in size_did_list:
        predicates = ' '.join(['AND dataset_id={}'.format(did), 'AND algorithm_id={}'.format(aid)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            dict_param = str2dict(row['parameter'])
            dict_rslt = str2dict(row['result'])
            bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
            idx_impl = dict_param['index_impl']

            output = {}
            output['n'] = int(dict_rslt['Dataset_numIndexed'])
            output['theta'] = float(dict_param['theta'])
            output['setting'] = get_setting_label(bLF, bPF, idx_impl)
            if output['setting'] is None: continue
            output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
            output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_qlen(data_info, alg_name=alg_name_final):
    dname, size, nr, qlen = parse_data_info(data_info)
    qlen_did_list = get_did_list(data_info)
    qlen_list = list(map(lambda x:int(x[0]), qlen_did_list))
    aid = dict_alg[alg_name]
    output_list = []
    for qlen, did in qlen_did_list:
        predicates = ' '.join(['AND dataset_id={}'.format(did), 'AND algorithm_id={}'.format(aid)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            dict_param = str2dict(row['parameter'])
            dict_rslt = str2dict(row['result'])
            bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
            idx_impl = dict_param['index_impl']

            output = {}
            output['qlen'] = int(qlen)
            output['theta'] = float(dict_param['theta'])
            output['setting'] = get_setting_label(bLF, bPF, idx_impl)
            if output['setting'] is None: continue
            output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
            output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
            output['Time_IndexFilter'] = float(dict_rslt['Time_QS_IndexFilter']) + float(dict_rslt['Time_TS_IndexFilter'])
            output['Num_Verified'] = int(dict_rslt['Num_QS_Verified']) + int(dict_rslt['Num_TS_Verified'])
            output['Len_Retrieved'] = int(dict_rslt['Len_QS_Retrieved']) + int(dict_rslt['Len_TS_Retrieved'])
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df    

In [4]:
def df_time_by_nr(data_info, alg_name=alg_name_final):
    assert type(data_info) == DataInfo
    dname, size, nr, qlen = data_info
    nr_did_list = get_did_list(data_info)
    aid = dict_alg[alg_name]
    output_list = []
    for nr, did in nr_did_list:
        predicates = ' '.join(['AND dataset_id={}'.format(did), 'AND algorithm_id={}'.format(aid)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            dict_param = str2dict(row['parameter'])
            dict_rslt = str2dict(row['result'])
            bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
            idx_impl = dict_param['index_impl']

            output = {}
            output['nr'] = int(nr)
            output['theta'] = float(dict_param['theta'])
            output['setting'] = get_setting_label(bLF, bPF, idx_impl)
            if output['setting'] is None: continue
            output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
            output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def plot_time_by_filtering(data_info, alg_name=alg_name_final):
    df = df_time_by_filtering(data_info, alg_name=alg_name_final)
    f, axes = plt.subplots(1, 3, figsize=(21, 6), sharex=False, sharey=False)
    sb.catplot(x='theta', y='Time_Total', hue='setting', data=df, kind='bar', ax=axes[0])
    sb.catplot(x='theta', y='Time_SearchPerQuery_MEAN', hue='setting', data=df, kind='bar', ax=axes[1])
    sb.catplot(x='theta', y='Num_Verified', hue='setting', data=df, kind='bar', ax=axes[2])
    axes[0].set(ylabel='Total Time (sec)', yscale='log')
    axes[1].set(ylabel='Mean query time (s)', yscale='log')
    axes[2].set(ylabel='# Verified', yscale='log')
    for i in range(2,5): plt.close(i)

In [ ]:
def plot_time_by_size(data_info, alg_name=alg_name_final):
    df = df_time_by_size(data_info, alg_name=alg_name_final)
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
#     for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
#         sb.lineplot(data=df[df.theta==theta], x="n", y="Time_Total", hue="setting", marker='o', ax=axes[0,i])
#         axes[0,i].set(ylabel='Total Time (sec)', xscale='log', yscale='log')
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="n", y="Time_SearchPerQuery_MEAN", hue="setting", marker='o', ax=axes[i])
        axes[i].set(ylabel='Mean query time (s)', xscale='log', yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()

In [ ]:
def plot_time_by_qlen(data_info, alg_name=alg_name_final, y='Time_SearchPerQuery_MEAN', ylabel='Mean query time (s)'):
    dname, size, nr, qlen = parse_data_info(data_info)
    qlen_list = list(map(lambda x:int(x[0]), get_did_list(data_info)))
    df = df_time_by_qlen(data_info, alg_name=alg_name_final)
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
#     for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
#         sb.lineplot(data=df[df.theta==theta], x="n", y="Time_Total", hue="setting", marker='o', ax=axes[0,i])
#         axes[0,i].set(ylabel='Total Time (sec)', xscale='log', yscale='log')
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="qlen", y=y, hue="setting", marker='o', ax=axes[i])
        axes[i].set(xticks=qlen_list, ylabel=ylabel, yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()

In [ ]:
def plot_time_by_nr(data_info, alg_name=alg_name_final):
    df = df_time_by_nr(data_info, alg_name=alg_name_final)
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
#     for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
#         sb.lineplot(data=df[df.theta==theta], x="n", y="Time_Total", hue="setting", marker='o', ax=axes[0,i])
#         axes[0,i].set(ylabel='Total Time (sec)', xscale='log', yscale='log')
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="nr", y="Time_SearchPerQuery_MEAN", hue="setting", marker='o', ax=axes[i])
        axes[i].set(ylabel='Mean query time (s)', xscale='log', yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()

In [ ]:
def compare_alg(alg0_name, alg1_name):
    aid0 = dict_alg[alg0_name]
    aid1 = dict_alg[alg1_name]
    predicates = ' '.join(['AND (algorithm_id={} OR algorithm_id={})'.format(aid0, aid1)])
    query = query_tmpl.format(predicates)
    output_list = []
    for row in execute_query(query):
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])
        bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
        idx_impl = dict_param['index_impl']

        output = {}
        output['dataset'] = dict_rslt['Dataset_Name']
        output['size'] = dict_rslt['Dataset_Name']
        output['alg'] = dict_rslt['Alg_Name']+'_'+dict_rslt['Alg_Version']
        output['theta'] = float(dict_param['theta'])
        output['setting'] = get_setting_label(bLF, bPF, idx_impl)
        if output['setting'] is None: continue
        output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
        output['Time_Search'] = (float(dict_rslt['Time_TSTotal']) + float(dict_rslt['Time_QSTotal']))/1000
        output['Num_Verified'] = int(dict_rslt['Num_QS_Verified']) + int(dict_rslt['Num_TS_Verified'])
        output['Num_QS_Result'] = int(dict_rslt['Num_QS_Result'])
        output['Num_TS_Result'] = int(dict_rslt['Num_TS_Result'])
        
        output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=['NoFilter', 'IF', 'ICF', 'NoIndex', 'NaivePF', 'IPF', 'LF', 'PF'])
    
    df0 = df[df.alg==alg0_name].drop(['alg'], axis=1)
    df1 = df[df.alg==alg1_name].drop(['alg'], axis=1)
    df_merged = pd.merge(df0, df1, on=['dataset', 'size', 'theta', 'setting'], how='outer', suffixes=['_0','_1'])
    df_merged['diff_Time'] = df_merged['Time_Total_0'] - df_merged['Time_Total_1']
    df_merged['diff_Time_ratio'] = (df_merged['Time_Total_0'] - df_merged['Time_Total_1'])/df_merged['Time_Total_0']
    df_merged['diff_Time_Search'] = df_merged['Time_Search_0'] - df_merged['Time_Search_1']
    df_merged['diff_Time_Search_ratio'] = (df_merged['Time_Search_0'] - df_merged['Time_Search_1'])/df_merged['Time_Search_0']
    df_merged['diff_Verify'] = df_merged['Num_Verified_0'] - df_merged['Num_Verified_1']
    df_merged['diff_Verify_ratio'] = (df_merged['Num_Verified_0'] - df_merged['Num_Verified_1'])/df_merged['Num_Verified_0']
    df_merged['diff_QS_Result'] = df_merged['Num_QS_Result_0'] - df_merged['Num_QS_Result_1']
    df_merged['diff_TS_Result'] = df_merged['Num_TS_Result_0'] - df_merged['Num_TS_Result_1']
    print('diff_Time:',df_merged['diff_Time'].mean())
    print('diff_Time_ratio:',df_merged['diff_Time_ratio'].mean())
    print('diff_Time_Search:',df_merged['diff_Time_Search'].mean())
    print('diff_Time_Search_ratio:',df_merged['diff_Time_Search_ratio'].mean())
    print('diff_Verify:',df_merged['diff_Verify'].mean())
    print('diff_Verify_ratio:',df_merged['diff_Verify_ratio'].mean())
    print('diff_QS_Result:',df_merged['diff_QS_Result'].mean())
    print('diff_TS_Result:',df_merged['diff_TS_Result'].mean())
    df_merged = df_merged.reindex(sorted(df_merged.columns), axis=1)
    return df_merged